# Build the Conversational Search Building Blocks

<div>
<img src="module1/all_components.png", width="800"/>
</div>



### In this lab, We will build the above components one by one to design an end to end conversational search application where you can simply upload a pdf and ask questions over the pdf content. The components include,

* **OpenSearch** as the Vector Database
* **Sagemaker endpoints** to host Embedding and the large language models
* **DynamoDB** as the memory store
* **Lambda functions** as the Document and Query Enoders
* **Ec2 instance** to host the web application

---

The lab includes the following steps:

1. [Get the Cloudformation outputs](#Get-the-Cloudformation-outputs)
2. [Component 1 : OpenSearch Vector DB](#Component-1-:-OpenSearch-Vector-DB)
3. [Component 2 : Embedding and LLM Endpoints](#Component-2-:--Embedding-and-LLM-Endpoints)
4. [Component 3 : Memory Store](#Component-3-:--Memory-Store)
5. [Component 4 : Document and Query Encoder](#Component-4-:--Document-and-Query-Encoder)
    - [4.1 Package the dependant libraries and handler files for lambda functions](#4.1-Package-the-dependant-libraries-and-handler-files-for-lambda-functions)
    - [4.2 Create the IAM role for Lambda](#4.2-Create-the-IAM-role-for-Lambda)
    - [4.3 Deploy lambda functions](#4.3-Deploy-lambda-functions)
    - [4.4 Create external URL for queryEncoder Lambda](#4.4-Create-external-URL-for-queryEncoder-Lambda)
6. [Component 5 : Client WebServer](#Component-5-:-Client-WebServer)


## Get the Cloudformation outputs

In [ ]:
import sagemaker, boto3, json, time
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

cfn = boto3.client('cloudformation')
response = cfn.list_stacks(
   StackStatusFilter=['CREATE_COMPLETE']
)
for cfns in response['StackSummaries']:
    if('semantic-search' in cfns['StackName']):
        stackname = cfns['StackName']
stackname

cfn_outputs = cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']
env_variables = {"aws_region":aws_region}
cfn_outputs

## Component 1 : OpenSearch Vector DB

<div>
<img src="module1/vectordb.png" width="600"/>
</div>

In [ ]:
for output in cfn_outputs:
    if('opensearch' in output['OutputKey'].lower()):
        env_variables[output['OutputKey']] = output['OutputValue']
        
opensearch_ = boto3.client('opensearch')

response = opensearch_.describe_domain(
    DomainName=env_variables['OpenSearchDomainName']
)

print("OpenSearch Version: "+response['DomainStatus']['EngineVersion']+"\n")
print("OpenSearch Configuration\n------------------------\n")
print(json.dumps(response['DomainStatus']['ClusterConfig'], indent=4))        

## Component 2 : Embedding and LLM Endpoints


<div>
<img src="module1/ml_models.png" width="600"/>
</div>

In [ ]:
sagemaker_ = boto3.client('sagemaker')

for output in cfn_outputs:
    if('endpointname' in output['OutputKey'].lower()):
        env_variables[output['OutputKey']] = output['OutputValue']
        print(output['OutputKey'] + " : "+output['OutputValue']+"\n"+"------------------------------------------------")
        print(json.dumps(sagemaker_.describe_endpoint_config(EndpointConfigName = sagemaker_.describe_endpoint(
    EndpointName=output['OutputValue']
                            )['EndpointConfigName'])['ProductionVariants'][0],indent = 4))
                        

## Component 3 : Memory Store

<div>
<img src="module1/memory.png" width="600"/>
</div>

In [ ]:
dynamo = boto3.client('dynamodb')

response = dynamo.create_table(
    TableName='conversation-history-memory',
    AttributeDefinitions=[
        {
            'AttributeName': 'SessionId',
            'AttributeType': 'S',
        }
    ],
    KeySchema=[
        {
            'AttributeName': 'SessionId',
            'KeyType': 'HASH',
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5,
    }
)
env_variables['DynamoDBTableName'] = response['TableDescription']['TableName']

print("dynamo DB Table, '"+response['TableDescription']['TableName']+"' is created")


## Component 4 : Document and Query Encoder

<div>
<img src="module1/encoders.png" width="600"/>
</div>

### 4.1 Package the dependant libraries and handler files for lambda functions

In [ ]:
from IPython.utils import io

with io.capture_output() as captured:

    # Download the Langchain module
    !aws s3 cp s3://ws-assets-prod-iad-r-gru-527b8c19222c1182/2108cfcf-6cd6-4613-83c0-db4e55998757/Langchain.zip .

    # Create a folder for DocumentEncoder Lambda and unzip the Langchain.zip contents
    !mkdir -p documentEncoder
    !unzip -o Langchain.zip -d documentEncoder/
    !cp -f chain_documentEncoder.py main_documentEncoder.py documentEncoder/
    !cd documentEncoder && zip -r documentEncoder.zip *

    # Create a folder for QueryEncoder Lambda and unzip the Langchain.zip contents
    !mkdir -p queryEncoder
    !unzip -o Langchain.zip -d queryEncoder/
    !cp -f chain_queryEncoder.py main_queryEncoder.py queryEncoder/
    !cd queryEncoder && zip -r queryEncoder.zip *

#Push Lambda artefacts to s3 bucket
for output in cfn_outputs:
    if('s3' in output['OutputKey'].lower()):
        s3_bucket = output['OutputValue']


!aws s3 cp documentEncoder/documentEncoder.zip s3://$s3_bucket
!aws s3 cp queryEncoder/queryEncoder.zip s3://$s3_bucket
    
print("\ndocumentEncoder.zip and queryEncoder.zip pushed to "+s3_bucket)

### 4.2 Create the IAM role for Lambda

In [ ]:
iam_ = boto3.client('iam')

lambda_iam_role = iam_.create_role(
    RoleName='LambdaRoleforSearch',
    AssumeRolePolicyDocument='{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]}',
    Description='LLMApp Lambda Permissions',
    
)

policies = [
    'AmazonDynamoDBFullAccess',
    'AmazonSagemakerFullAccess',
    'AmazonS3FullAccess',
    'AmazonOpenSearchServiceFullAccess',
    'CloudWatchLogsFullAccess',
    'SecretsManagerReadWrite']

for policy in policies:
    iam_.attach_role_policy(
        RoleName='LambdaRoleforSearch',
        PolicyArn='arn:aws:iam::aws:policy/'+policy
    )
time.sleep(5)    
lambda_iam_role_arn = lambda_iam_role['Role']['Arn']
lambda_iam_role_arn

### 4.3 Deploy lambda functions

In [ ]:
lambda_ = boto3.client('lambda')

encoders = ['queryEncoder','documentEncoder']

for encoder in encoders:
    response = lambda_.create_function(
    FunctionName=encoder,
    Runtime='python3.9',
    Role=lambda_iam_role_arn,
    Handler='main_'+encoder+'.handler',
    Code={
        
        'S3Bucket': s3_bucket,
        'S3Key': encoder+'.zip',
        
    },
    Timeout=900,
    MemorySize=512,
    Environment={
        'Variables': env_variables
    }
    )
    print(response['FunctionArn'])
    documentEncoder = response['FunctionArn']

### 4.4 Create external URL for queryEncoder Lambda

In [ ]:
response = lambda_.add_permission(
    FunctionName='queryEncoder',
    StatementId='queryEncoder_permissions',
    Action="lambda:InvokeFunctionUrl",
    Principal=documentEncoder.split(':')[4],
    FunctionUrlAuthType='AWS_IAM'
)

response = lambda_.create_function_url_config(
    FunctionName='queryEncoder',
    AuthType='AWS_IAM',
    Cors={
        'AllowCredentials': True,

        'AllowMethods':["*"],
        'AllowOrigins': ["*"]

    },
    InvokeMode='RESPONSE_STREAM'
)

query_invoke_URL = response['FunctionUrl']
query_invoke_URL

## Component 5 : Client WebServer

<div>
<img src="module1/webserver.png" width="600"/>
</div>

### 5.1 Update the webapp code artefacts 

In [ ]:
#modify the code files with lambda url and s3 bucket names
query_invoke_URL_cmd = query_invoke_URL.replace("/","\/")
!sed -i 's/API_URL_TO_BE_REPLACED/{query_invoke_URL_cmd}/g' webapp/api.py
!sed -i 's/pdf-repo-uploads/{s3_bucket}/g' webapp/app.py

#push the webapp code archive to s3
!cd webapp && zip -r ../webapp.zip *
!aws s3 cp webapp.zip s3://$s3_bucket
    
response = cfn.describe_stack_resources(
    StackName=stackname
)
for resource in response['StackResources']:
    if(resource['ResourceType'] == 'AWS::EC2::Instance'):
        ec2_instance_id = resource['PhysicalResourceId']


print("\nec2_instance_id: "+ec2_instance_id)

### 5.2 Execute ssh comands in ec2 terminal to start the app

In [ ]:
# function to execute commands in ec2 terminal
def execute_commands_on_linux_instances(client, commands):
    resp = client.send_command(
        DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
        Parameters={'commands': commands},
        InstanceIds=[ec2_instance_id],
    )
    return resp['Command']['CommandId']


ssm_client = boto3.client('ssm') 


commands = [
            'aws s3 cp s3://'+s3_bucket+'/webapp.zip /home/ec2-user/',
            'unzip -o /home/ec2-user/webapp.zip -d /home/ec2-user/'  ,      
            'sudo chmod -R 0777 /home/ec2-user/',
            'aws s3 cp /home/ec2-user/pdfs s3://'+s3_bucket+'/sample_pdfs/ --recursive',
            'python3 -m venv /home/ec2-user/.myenv',
            'source /home/ec2-user/.myenv/bin/activate',
            'pip install streamlit',
            'pip install boto3',
    
            #start the web applicaiton
            'streamlit run /home/ec2-user/app.py',
            
            ]

command_id = execute_commands_on_linux_instances(ssm_client, commands)

time.sleep(3)

response = ssm_client.get_command_invocation(
    CommandId=command_id,
    InstanceId=ec2_instance_id
)
print(response['Status'])
print('---------')
print(response['StandardOutputContent']+" The webserver is up and running")


### 5.3 check the ec2 ports which runs the app

In [ ]:
#check the status of the applicaiton
commands = [
            "sudo lsof -i -P -n | grep streamlit"
             ]

command_id = execute_commands_on_linux_instances(ssm_client, commands)

time.sleep(5)

response = ssm_client.get_command_invocation(
    CommandId=command_id,
    InstanceId=ec2_instance_id
)
print(response['Status'])
print('---------')
print(response['StandardOutputContent'])
print(response['StandardErrorContent'])


### 5.4 Open the app using ec2 public DNS

In [ ]:
port_num = response['StandardOutputContent'].replace(" ","").split("(LISTEN)")[0].split(":")[1]
ec2_ = boto3.client('ec2')
response = ec2_.describe_instances(
    InstanceIds=[ec2_instance_id]
)
public_ip = response['Reservations'][0]['Instances'][0]['PublicIpAddress']
print("Click the URL to open the application")
print('http://'+public_ip+":"+port_num)